In [1]:
import pyspark
from  pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.conf import SparkConf

In [2]:
# https://stackoverflow.com/a/41946447/2049763
# https://www.edureka.co/community/5268/how-to-change-the-spark-session-configuration-in-pyspark
sc.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.sql.execution.arrow.enabled', 'true'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.app.name', 'Jupyter Notebook #1'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.port', '41393'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1573796736499'),
 ('spark.eventLog.dir', '/var/log/ipnb'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.deployMode', 'client'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.sql.warehouse.dir', '/vagrant/hive/warehouse')]

In [3]:
# https://stackoverflow.com/a/39528114/2049763
# https://stackoverflow.com/q/43692453/2049763

config = SparkConf()
config.setMaster('spark://192.72.33.100:7077')
config.set("spark.driver.memory", "3g") 
config.set("spark.executor.memory", "2g") 
config.set("spark.network.timeout", "180")
config.set("spark.driver.host", "192.72.33.100")
config.set("spark.local.ip", "192.72.33.100")
config.set("spark.driver.bindAddress", "192.72.33.100")
config.setAppName('my_coco_app')

In [ ]:
sc.stop()
sc = SparkContext(conf=config)

In [ ]:
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc.getConf().getAll()

#### Required part finished
* The end go to [work](#Real-work-starts-here:)
* skip practice part

In [6]:
sqlContext = SQLContext(spark)

In [21]:
# https://pythonbasics.org/read-json-file/
import json

# read file
with open("/vagrant/datasets/coco/instances_val2014.json", 'r') as myfile:
    data=myfile.read()

In [22]:
# parse file
obj = json.loads(data)

In [9]:
for key in obj.keys(): 
    print(key)


info
images
licenses
annotations
categories


In [10]:
obj["categories"]

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

In [11]:
# https://stackoverflow.com/a/49399359/2049763
# https://stackoverflow.com/a/49676143/2049763
categories = spark.read.json(sc.parallelize(obj["categories"]))

In [12]:
categories.head()

Row(id=1, name='person', supercategory='person')

In [13]:
categories.show()

+---+-------------+-------------+
| id|         name|supercategory|
+---+-------------+-------------+
|  1|       person|       person|
|  2|      bicycle|      vehicle|
|  3|          car|      vehicle|
|  4|   motorcycle|      vehicle|
|  5|     airplane|      vehicle|
|  6|          bus|      vehicle|
|  7|        train|      vehicle|
|  8|        truck|      vehicle|
|  9|         boat|      vehicle|
| 10|traffic light|      outdoor|
| 11| fire hydrant|      outdoor|
| 13|    stop sign|      outdoor|
| 14|parking meter|      outdoor|
| 15|        bench|      outdoor|
| 16|         bird|       animal|
| 17|          cat|       animal|
| 18|          dog|       animal|
| 19|        horse|       animal|
| 20|        sheep|       animal|
| 21|          cow|       animal|
+---+-------------+-------------+
only showing top 20 rows



In [14]:
images = spark.read.json(sc.parallelize(obj["images"]))

In [15]:
images.show()

+--------------------+-------------------+--------------------+--------------------+------+------+-------+-----+
|            coco_url|      date_captured|           file_name|          flickr_url|height|    id|license|width|
+--------------------+-------------------+--------------------+--------------------+------+------+-------+-----+
|http://images.coc...|2013-11-14 11:18:45|COCO_val2014_0000...|http://farm9.stat...|   360|391895|      3|  640|
|http://images.coc...|2013-11-14 11:38:44|COCO_val2014_0000...|http://farm1.stat...|   480|522418|      4|  640|
|http://images.coc...|2013-11-14 12:36:29|COCO_val2014_0000...|http://farm3.stat...|   336|184613|      3|  500|
|http://images.coc...|2013-11-14 13:02:53|COCO_val2014_0000...|http://farm5.stat...|   640|318219|      3|  556|
|http://images.coc...|2013-11-14 16:03:19|COCO_val2014_0000...|http://farm5.stat...|   640|554625|      3|  426|
|http://images.coc...|2013-11-14 17:02:52|COCO_val2014_0000...|http://farm7.stat...|   427|39713

In [23]:
annotations = spark.read.json(sc.parallelize(obj["annotations"]))

In [28]:
# https://stackoverflow.com/a/37400892/2049763
annotations_schema = annotations.schema; annotations_schema

StructType(List(StructField(area,DoubleType,true),StructField(bbox,ArrayType(DoubleType,true),true),StructField(category_id,LongType,true),StructField(id,LongType,true),StructField(image_id,LongType,true),StructField(iscrowd,LongType,true),StructField(segmentation,StringType,true)))

In [35]:
# https://stackoverflow.com/q/41386557/2049763
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, ArrayType

annotations_schema = StructType([
StructField('area',StringType(), True),
StructField('bbox',ArrayType(DoubleType(),True),True),
StructField('category_id',LongType(),True),
StructField('id',LongType(),True),
StructField('image_id',LongType(),True),
StructField('iscrowd',LongType(),True),
StructField('segmentation',StringType(),True)
])


In [17]:
annotations.orderBy("id").show()

+------------------+--------------------+-----------+---+--------+-------+--------------------+
|              area|                bbox|category_id| id|image_id|iscrowd|        segmentation|
+------------------+--------------------+-----------+---+--------+-------+--------------------+
|       148769.3913|[1.23, 210.14, 60...|         58| 87|   34567|      0|[[14.75,255.61,49...|
|18809.937250000006|[9.83, 147.47, 14...|         58|131|  135167|      0|[[29.49,172.05,29...|
|       22288.89475|[286.52, 181.27, ...|         58|144|  381318|      0|[[340.82,353.35,3...|
|2765.1486500000005|[199.84, 200.46, ...|         58|156|  558840|      0|[[239.97,260.24,2...|
|19232.358400000005|[107.52, 271.02, ...|         58|202|  157883|      0|[[203.52,390.06,1...|
|       64170.58415|[64.75, 156.54, 4...|         58|212|  292995|      0|[[68.16,257.08,14...|
|19358.362200000003|[290.12, 78.07, 2...|         58|257|   43494|      0|[[573.49,89.64,56...|
|17809.733500000002|[38.22, 338.96, 2...

In [18]:
print((annotations.count(), len(annotations.columns)))

(291875, 7)


In [19]:
# https://stackoverflow.com/a/47046277/2049763
annotations.select('image_id').distinct().count()

40137

In [28]:
# from pyspark.sql import Window
# w = Window.partitionBy('category_id')

In [29]:
# annotations.count().over(w)

In [30]:
category_id_count_df = annotations.groupBy('category_id').count()

In [ ]:
category_id_count_df.show(100, truncate = False) 

In [ ]:
import shutil
shutil.rmtree("/vagrant/datasets/coco/instances_val2014_category_outer_join_df.csv") 

In [ ]:
# https://stackoverflow.com/a/42111775/2049763
category_id_count_df.repartition(1).write.csv(path="/vagrant/datasets/coco/instances_val2014.csv", mode="append", header="true")

In [51]:
category_inner_join_df = category_id_count_df.join(categories, category_id_count_df.category_id == categories.id)
category_inner_join_df.show()

+-----------+-----+---+-------------+-------------+
|category_id|count| id|         name|supercategory|
+-----------+-----+---+-------------+-------------+
|         65| 1450| 65|          bed|    furniture|
|         19| 2194| 19|        horse|       animal|
|         54| 1461| 54|     sandwich|         food|
|         22| 1863| 22|     elephant|       animal|
|          7| 1602|  7|        train|      vehicle|
|         77| 2236| 77|   cell phone|   electronic|
|         34|  935| 34|      frisbee|       sports|
|         50| 2131| 50|        spoon|      kitchen|
|         57| 2684| 57|       carrot|         food|
|         32| 2253| 32|          tie|    accessory|
|         43| 1626| 43|tennis racket|       sports|
|         84| 8561| 84|         book|       indoor|
|         31| 4116| 31|      handbag|    accessory|
|         39| 1022| 39| baseball bat|       sports|
|         25| 1767| 25|      giraffe|       animal|
|          6| 2027|  6|          bus|      vehicle|
|         72

In [52]:
category_inner_join_df.repartition(1).write.csv(path="/vagrant/datasets/coco/instances_val2014_category_inner_join_df.csv", mode="append", header="true")

In [55]:
# http://www.learnbymarketing.com/1100/pyspark-joins-by-example/
category_outer_join_df = category_id_count_df.join(categories, category_id_count_df.category_id == categories.id, how='full')


category_outer_join_df.repartition(1).write.csv(path="/vagrant/datasets/coco/instances_val2014_category_outer_join_df.csv", mode="append", header="true")

In [59]:
# https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.orderBy
category_outer_join_df = category_id_count_df.join(categories, category_id_count_df.category_id == categories.id, how='full')

In [62]:
category_outer_join_df = category_outer_join_df.sort("category_id", ascending=True)

In [65]:
category_outer_join_df.repartition(1).write.csv(path="/vagrant/datasets/coco/instances_val2014_category_outer_join_df.csv", mode="append", header="true")

## Real work starts here:

### instances_train2014

In [13]:
# read file
path = "/vagrant/datasets/coco/instances_train2014.json"
# raw_data = sc.textFile(path)

In [14]:
import json

In [15]:
# read file
with open(path, 'r') as myfile:
    data=myfile.read()

In [16]:
# parse file
obj = json.loads(data)

In [17]:
# parse file
# instances_train2014 = raw_data.map(json.loads)
# instances_train2014.persist()

In [47]:
from pyspark import sql

In [19]:
categories = spark.createDataFrame(sql.Row(**x) for x in obj['categories'])
categories.show()

+---+-------------+-------------+
| id|         name|supercategory|
+---+-------------+-------------+
|  1|       person|       person|
|  2|      bicycle|      vehicle|
|  3|          car|      vehicle|
|  4|   motorcycle|      vehicle|
|  5|     airplane|      vehicle|
|  6|          bus|      vehicle|
|  7|        train|      vehicle|
|  8|        truck|      vehicle|
|  9|         boat|      vehicle|
| 10|traffic light|      outdoor|
| 11| fire hydrant|      outdoor|
| 13|    stop sign|      outdoor|
| 14|parking meter|      outdoor|
| 15|        bench|      outdoor|
| 16|         bird|       animal|
| 17|          cat|       animal|
| 18|          dog|       animal|
| 19|        horse|       animal|
| 20|        sheep|       animal|
| 21|          cow|       animal|
+---+-------------+-------------+
only showing top 20 rows



In [9]:
# categoriesRDD = sc.parallelize(obj["categories"])

In [10]:
# categories = spark.read.json(categoriesRDD)

In [38]:
# https://stackoverflow.com/q/41386557/2049763
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, ArrayType

annotations_schema = StructType([
StructField('area',StringType(), True),
StructField('bbox',ArrayType(StringType(),True),True),
StructField('category_id',LongType(),True),
StructField('id',LongType(),True),
StructField('image_id',LongType(),True),
StructField('iscrowd',LongType(),True),
StructField('segmentation',StringType(),True)
])

In [39]:
annotations = spark.createDataFrame( (sql.Row(**x) for x in obj['annotations']), schema=annotations_schema)
annotations.show()

+------------------+--------------------+-----------+----+--------+-------+--------------------+
|              area|                bbox|category_id|  id|image_id|iscrowd|        segmentation|
+------------------+--------------------+-----------+----+--------+-------+--------------------+
|2765.1486500000005|[199.84, 200.46, ...|         58| 156|  558840|      0|[[239.97, 260.24,...|
|1545.4213000000007|[234.22, 317.11, ...|         58| 509|  200365|      0|[[247.71, 354.7, ...|
| 5607.661349999996|[239.48, 347.87, ...|         58| 603|  200365|      0|[[274.58, 405.68,...|
|               0.0|[296.65, 388.33, ...|         58| 918|  200365|      0|[[296.65, 388.33,...|
| 800.4132499999978|[251.87, 333.42, ...|         58|1072|  200365|      0|[[251.87, 356.13,...|
|2243.7513000000004|[337.02, 244.46, ...|         18|1727|  495357|      0|[[374.46, 310.42,...|
| 1698.440800000001|[213.81, 192.39, ...|         18|1728|  116061|      0|[[214.59, 205.04,...|
|19957.726749999994|[324.66, 2

In [11]:
# annotationsRDD = sc.parallelize(obj["annotations"])

In [ ]:
# annotations = spark.read.json(annotationsRDD)

In [40]:
print((annotations.count(), len(annotations.columns)))
print(annotations.select('image_id').distinct().count())

(291875, 7)
40137


In [41]:
category_id_count_df = annotations.groupBy('category_id').count()

In [42]:
category_outer_join_df = category_id_count_df.join(categories, category_id_count_df.category_id == categories.id, how='full')
category_outer_join_df = category_outer_join_df.sort("category_id", ascending=True)

In [43]:
import shutil

In [45]:
instances_train2014_csv_path = "/vagrant/datasets/coco/instances_train2014_category_outer_join_df.csv"
if os.path.isdir(instances_train2014_csv_path):
    shutil.rmtree(instances_train2014_csv_path) 

In [46]:
category_outer_join_df.repartition(1).write.csv(path=instances_train2014_csv_path, mode="append", header="true")

### instances_train2017

In [6]:
# read file
path = "/vagrant/datasets/coco/instances_train2017.json"
with open(path, 'r') as myfile:
    data=myfile.read()

In [7]:
import json

In [8]:
# parse file
obj = json.loads(data)

In [9]:
from pyspark import sql

In [10]:
categories_train2017 = spark.createDataFrame(sql.Row(**x) for x in obj['categories'])

In [11]:
# https://stackoverflow.com/q/41386557/2049763
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, ArrayType

annotations_schema = StructType([
StructField('area',StringType(), True),
StructField('bbox',ArrayType(StringType(),True),True),
StructField('category_id',LongType(),True),
StructField('id',LongType(),True),
StructField('image_id',LongType(),True),
StructField('iscrowd',LongType(),True),
StructField('segmentation',StringType(),True)
])

In [ ]:
annotations_train2017 = spark.createDataFrame( (sql.Row(**x) for x in obj['annotations']), schema=annotations_schema)

In [ ]:
print((annotations_train2017.count(), len(annotations_train2017.columns)))
print(annotations_train2017.select('image_id').distinct().count())

In [ ]:
category_id_count_df_train2017 = annotations_train2017.groupBy('category_id').count()

In [ ]:
category_outer_join_df_train2017 = category_id_count_df_train2017.join(
    categories_train2017, category_id_count_df_train2017.category_id == categories_train2017.id, how='full')
category_outer_join_df_train2017 = category_outer_join_df_train2017.sort("category_id", ascending=True)

In [ ]:
import shutil

In [ ]:
instances_train2017_csv_path = "/vagrant/datasets/coco/instances_train2017_category_outer_join_df.csv"
if os.path.isdir(instances_train2017_csv_path):
    shutil.rmtree(instances_train2017_csv_path) 

In [ ]:
category_outer_join_df_train2017.repartition(1).write.csv(path=instances_train2017_csv_path, mode="append", header="true")